# **Few Shot Learning**

In [21]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np

In [ ]:
tfd = tfp.distributions

def generate_few_shot_data(num_classes, num_samples_per_class, input_dim):
    data = []
    for i in range(num_classes):
        x = np.random.rand(num_samples_per_class, input_dim)
        y = [i] * num_samples_per_class
        data.append((x, y))
    return data

class BayesianImageClassifier(tf.Module):
    def __init__(self, input_dim, output_dim):
        self.fc1 = tf.keras.layers.Dense(64, activation='relu', input_shape=(input_dim,))
        self.fc2 = tf.keras.layers.Dense(output_dim)
        self.prior = tfd.Normal(loc=0., scale=1.)

    def __call__(self, x, samples=1):
        outputs = []
        for _ in range(samples):
            h1 = self.fc1(x)
            out = self.fc2(h1)
            outputs.append(out)
        return outputs

def create_bayesian_meta_learning_model(num_classes, input_dim):
    model = BayesianImageClassifier(input_dim, num_classes)
    optimizer = tf.optimizers.Adam(learning_rate=0.001)
    return model, optimizer

def train_few_shot_learning(model, optimizer, data, num_epochs, num_samples_per_class):
    for epoch in range(num_epochs):
        for task_data in data:
            x, y = task_data
            x = tf.convert_to_tensor(x, dtype=tf.float32)
            y = tf.convert_to_tensor(y, dtype=tf.int32)
            task_input = x
            with tf.GradientTape() as tape:
                task_params = model.prior.sample([x.shape[0]])
                task_params = tf.reshape(task_params, [-1, 1])
                task_input = tf.concat([task_input, task_params], axis=1)
                task_predictions = model(task_input, samples=1)
                loss = tf.losses.sparse_categorical_crossentropy(y, task_predictions[-1])
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))

def test_few_shot_learning(model, data, num_samples_per_class):
    accuracy = 0
    for task_data in data:
        x, y = task_data
        x = tf.convert_to_tensor(x, dtype=tf.float32)
        y = tf.convert_to_tensor(y, dtype=tf.int64)
        task_input = x
        task_params = model.prior.sample([x.shape[0]])
        task_params = tf.reshape(task_params, [-1, 1])
        task_input = tf.concat([task_input, task_params], axis=1)
        task_predictions = model(task_input, samples=10)
        predicted_labels = tf.argmax(tf.reduce_mean(task_predictions, axis=0), axis=1)
        accuracy += tf.reduce_sum(tf.cast(predicted_labels == y, dtype=tf.int32)) / num_samples_per_class
    return accuracy / len(data)

num_classes = 5
num_samples_per_class = 5
input_dim = 64
num_epochs = 50

few_shot_data = generate_few_shot_data(num_classes, num_samples_per_class, input_dim)


bayesian_model, optimizer = create_bayesian_meta_learning_model(num_classes, input_dim)
train_few_shot_learning(bayesian_model, optimizer, few_shot_data, num_epochs, num_samples_per_class)

accuracy = test_few_shot_learning(bayesian_model, few_shot_data, num_samples_per_class)
print("Few-Shot Learning Accuracy:", accuracy)
